实验实例：PBN Android AA测试 https://bi.learnings.ai/#/pm/5b84f58e689998000116d3fd/abtest/exp?domain=cms&expId=5f3dd792f54b8a000119e514  
实验时间：2020-08-22 ~ 2020-09-07

对比组：A1(Ua0) & A4(Ua3) 

取数及当前方法计算显著性的结果
select avg(fin), stddev(fin),count(*) from (SELECT
  _table_suffix as date, unique_user_id,sum(pic_finish_nums) as fin
FROM
  `paint-by-number-3c789.learnings_data_warehouse_android.analytics_dm_action_userPrimaryMetric_di_*`
WHERE
  _table_suffix BETWEEN '20200822'
  AND '20200907'
  and abtest_tag like '%Ua0%'
  group by 1,2)
  union all
  select avg(fin), stddev(fin),count(*)  from (SELECT
  _table_suffix as date, unique_user_id,sum(pic_finish_nums) as fin
FROM
  `paint-by-number-3c789.learnings_data_warehouse_android.analytics_dm_action_userPrimaryMetric_di_*`
WHERE
  _table_suffix BETWEEN '20200822'
  AND '20200907'
  and abtest_tag like '%Ua3%'
  group by 1,2)
  
  
  -- ua0：avg:5.359787175047511  std: 8.199395443755122   2005216
  -- ua3：avg:5.379056000143576  std: 8.272769690991419   2006102
  
 当前方式：人天为样本

 p: 0.019137153378226687 显著

In [ ]:
#当前方式显著性计算代码
import math
import scipy.stats as stats


def count( x1, x2, s1, s2, n1, n2, alpha=0.05):
# 计算显著性
    z = (x1 - x2) / pow(s1 ** 2 / n1 + s2 ** 2 / n2, 1 / 2)
    if z > 0:
        p = (1 - stats.norm.cdf(z)) * 2
    else:
        p = 2 * stats.norm.cdf(z)
    # 计算置信区间
    d = stats.norm.ppf(1 - alpha / 2) * pow(s1 ** 2 / n1 + s2 ** 2 / n2, 1 / 2)
    floor = -(x1 - x2 - d)
    ceil = -(x1 - x2 + d)
    return p

x1 = 5.359787175047511
x2 = 5.379056000143576
s1 = 8.199395443755122
s2 = 8.272769690991419
n1 = 2005216
n2 = 2006102

p = count( x1, x2, s1, s2, n1, n2, alpha=0.05)
print(p)

0.019137153378226687


当前方式2：人为样本

取数：

 (select AVG(Si) ,stddev(Si)  from (
  select unique_user_id,sum(fin) as Si  from (SELECT
  _table_suffix as date, unique_user_id,sum(pic_finish_nums) as fin
FROM
  `paint-by-number-3c789.learnings_data_warehouse_android.analytics_dm_action_userPrimaryMetric_di_*`
WHERE
  _table_suffix BETWEEN '20200822'
  AND '20200907'
  and abtest_tag like '%Ua0%'
  group by 1,2) GROUP BY 1))
  union all
   (select AVG(Si) ,stddev(Si)  from (
  select unique_user_id,sum(fin) as Si  from (SELECT
  _table_suffix as date, unique_user_id,sum(pic_finish_nums) as fin
FROM
  `paint-by-number-3c789.learnings_data_warehouse_android.analytics_dm_action_userPrimaryMetric_di_*`
WHERE
  _table_suffix BETWEEN '20200822'
  AND '20200907'
  and abtest_tag like '%Ua3%'
  group by 1,2) GROUP BY 1))

--Ua0	
--20.93998488078048
--49.48935978699663
--513254
--Ua3
--21.06078408028554
--50.086428276735084
--512371



p值：0.21923891930286177 不显著


In [ ]:
import math
import scipy.stats as stats


def count( x1, x2, s1, s2, n1, n2, alpha=0.05):
# 计算显著性
    z = (x1 - x2) / pow(s1 ** 2 / n1 + s2 ** 2 / n2, 1 / 2)
    if z > 0:
        p = (1 - stats.norm.cdf(z)) * 2
    else:
        p = 2 * stats.norm.cdf(z)
    # 计算置信区间
    d = stats.norm.ppf(1 - alpha / 2) * pow(s1 ** 2 / n1 + s2 ** 2 / n2, 1 / 2)
    floor = -(x1 - x2 - d)
    ceil = -(x1 - x2 + d)
    return p

x2 = 21.06078408028554
x1 = 20.93998488078048
s2 = 50.086428276735084
s1 = 49.48935978699663
n2 = 512371
n1 = 513254

p = count( x1, x2, s1, s2, n1, n2, alpha=0.05)
print(p)

0.21923891930286177


用新的方式来计算方差.

1.首先查出两组人数

(SELECT
  count(distinct unique_user_id)
FROM
  `paint-by-number-3c789.learnings_data_warehouse_android.analytics_dm_action_userPrimaryMetric_di_*`
WHERE
  _table_suffix BETWEEN '20200822'
  AND '20200907'
  and abtest_tag like '%Ua0%') 
union all
(SELECT
  count(distinct unique_user_id)
FROM
  `paint-by-number-3c789.learnings_data_warehouse_android.analytics_dm_action_userPrimaryMetric_di_*`
WHERE
  _table_suffix BETWEEN '20200822'
  AND '20200907'
  and abtest_tag like '%Ua3%') 

  -- Ua0：513254
  -- Ua3：512371

  2.其次根据公式分别提取出需要的数据
  Ua0:
   select AVG(Ni) as miuN1,AVG(Si) as miuS1,stddev(Ni) as sgmN1 ,stddev(Si) as sgmS1, COVAR_SAMP(Ni,Si) as cov1 from (
  select unique_user_id,count(fin)/513254 as Ni ,sum(fin)/513254 as Si  from (SELECT
  _table_suffix as date, unique_user_id,sum(pic_finish_nums) as fin
FROM
  `paint-by-number-3c789.learnings_data_warehouse_android.analytics_dm_action_userPrimaryMetric_di_*`
WHERE
  _table_suffix BETWEEN '20200822'
  AND '20200907'
  and abtest_tag like '%Ua0%'
  group by 1,2) GROUP BY 1)

-- miuN1=7.611959622073329E-6
-- sgmN1=7.5468716529489155E-6  
-- miuS1=4.0798483559367195E-5 
-- sgmS1=9.642274543792179E-5
-- cov1=4.811171895285024E-10

 Ua3:
 select AVG(Ni) as miuN2,AVG(Si) as miuS2,stddev(Ni) as sgmN2 ,stddev(Si) as sgmS2, COVAR_SAMP(Ni,Si) as cov2 from (
  select unique_user_id,count(fin)/513254 as Ni ,sum(fin)/513254 as Si  from (SELECT
  _table_suffix as date, unique_user_id,sum(pic_finish_nums) as fin
FROM
  `paint-by-number-3c789.learnings_data_warehouse_android.analytics_dm_action_userPrimaryMetric_di_*`
WHERE
  _table_suffix BETWEEN '20200822'
  AND '20200907'
  and abtest_tag like '%Ua3%'
  group by 1,2) GROUP BY 1)

-- miuN2=7.641593461155268E-6
-- sgmN2=7.5766824777271935E-6  
-- miuS2=4.110455915788714E-5 
-- sgmS2=9.775422160258047E-5
-- cov2=4.861059444738555E-10

In [ ]:
#通过delta-method来计算方差和置信区间
import scipy.stats as st
import math 

def dm(K,miuN,miuS,sgmN,sgmS,cov):
    vest = (sgmS ** 2) / (miuN ** 2) + (miuS ** 2) * (sgmN ** 2) / (miuN ** 4) - 2 * miuS * cov / (miuN ** 3)
    bc = miuS / (miuN ** 3) * (sgmN ** 2 / K1) - 1 / (miuN ** 2) * (cov / K)
    pest = miuS / miuN - 1 + bc
    y_floor = pest - st.norm.ppf(0.975) * pow(vest / K, 1/2)
    y_ceil1 = pest + st.norm.ppf(0.975) * pow(vest / K, 1/2)
    return {
        "方差：":vest,
        "置信区间:":[y_floor,y_ceil1]
    }

K1 =513254    
miuN1=7.611959622073329E-6
sgmN1=7.5468716529489155E-6  
miuS1=4.0798483559367195E-5 
sgmS1=9.642274543792179E-5
cov1=4.811171895285024E-10

R1 = dm(K1,miuN1,miuS1,sgmN1,sgmS1,cov1)
print("Ua0的方差和置信区间：",R1)


K2 =512371    
miuN2=7.641593461155268E-6
sgmN2=7.5766824777271935E-6  
miuS2=4.110455915788714E-5 
sgmS2=9.775422160258047E-5
cov2=4.861059444738555E-10

R2 = dm(K2,miuN2,miuS2,sgmN2,sgmS2,cov2)
print("Ua3的方差和置信区间：",R2)


Ua0的方差和置信区间： {'方差：': 99.68854437056926, '置信区间:': [4.332466051750653, 4.387096472157213]}
Ua3的方差和置信区间： {'方差：': 102.53302196438213, '置信区间:': [4.351324025278139, 4.40677608662517]}


运算结果：

Ua0的方差和置信区间： {'方差：': 99.68854437056926, '置信区间:': [4.332466051750653, 4.387096472157213]}

Ua3的方差和置信区间： {'方差：': 102.53302196438213, '置信区间:': [4.351324025278139, 4.40677608662517]}

从运算结果看，置信区间相交，因此变化不显著。
从公式看，计算置信区间时需要减1。但实际结果不减1更实际更相符。需要进一步研究



下面尝试用每组的方差代入Z检验来计算

方式1：样本数取PV的样本数进行计算

P值：0.05498671759929719 不显著

In [ ]:
import math
import scipy.stats as stats


def count( x1, x2, pest1, pest2, n1, n2, alpha=0.05):
# 计算显著性
    z = (x1 - x2) / pow(pest1 / n1 + pest2 / n2, 1 / 2)
    if z > 0:
        p = (1 - stats.norm.cdf(z)) * 2
    else:
        p = 2 * stats.norm.cdf(z)
    # 计算置信区间
    d = stats.norm.ppf(1 - alpha / 2) * pow(s1 ** 2 / n1 + s2 ** 2 / n2, 1 / 2)
    floor = -(x1 - x2 - d)
    ceil = -(x1 - x2 + d)
    return p

x1 = 5.359787175047511
x2 = 5.379056000143576
pest1 = 99.68854437056926
pest2 = 102.53302196438213
n1 = 2005216
n2 = 2006102

p = count( x1, x2, pest1, pest2, n1, n2, alpha=0.05)
print(p)

0.05498671759929719


方式2：样本数取UV的样本数进行计算。论文中给出的两组差的方差即此方式

P值：0.3318836407269048 不显著

In [ ]:
import math
import scipy.stats as stats


def count( x1, x2, pest1, pest2, n1, n2, alpha=0.05):
# 计算显著性
    z = (x1 - x2) / pow(pest1 / n1 + pest2 / n2, 1 / 2)
    if z > 0:
        p = (1 - stats.norm.cdf(z)) * 2
    else:
        p = 2 * stats.norm.cdf(z)
    # 计算置信区间
    d = stats.norm.ppf(1 - alpha / 2) * pow(s1 ** 2 / n1 + s2 ** 2 / n2, 1 / 2)
    floor = -(x1 - x2 - d)
    ceil = -(x1 - x2 + d)
    return p

x1 = 5.359787175047511
x2 = 5.379056000143576
pest1 = 99.68854437056926
pest2 = 102.53302196438213
n1 = 513254
n2 = 512371

p = count( x1, x2, pest1, pest2, n1, n2, alpha=0.05)
print(p)

0.3318836407269048
